In [1]:
from utils import parse_llm_output, make_input, extract_strings
from groq_model import Llama3_1_70B_Versatile, Llama3_1_8B_Instant

In [2]:
model = Llama3_1_70B_Versatile()

In [3]:

prompt = """You are an expert mental health professional. Your goal is to read patient's answers to some of the questions given and find out the category (or categories) they belong to and extract relevant phrases from the answer which can be useful to assess patient's mental state.

Task Instructions:
	- You can pick multiple categories which are most suitable for the answer.
	- Categories for each question are provided, do not create categories of your own.
    - All ways add number and the title of the question before generating categories.
    - Do not repeat answer in your response.
	- Enclose all the comma separated categories you generate with left bracket [ and right bracket ]
	- Enclose each the comma separated extracted phrases in left angle bracket < and right angle bracket >.
	

1. Noise during sleep 
Question: Can you tell me a little about the noise level in your sleep environment? Is it usually quiet, or are there disturbances like traffic or household noises?
Categories: [quiet, moderate noise, very noise]
Answer: I don't really pay attention to noise. It's always there. There's this constant hum in my head, so other noises don't really bother me.

2. Temperature During Sleep
Question: How would you describe the temperature in your bedroom when you sleep? Is it too warm, too cold, or just right?
Categories: [too cold, comfortable, too hot]
Answer: I don't know. I'm usually cold, even when the heat is on. Nothing seems to warm me up.

3. Sleep Continuity
Question: Do you tend to sleep through the night without waking up, or do you find yourself waking up several times? If so, can you tell me a bit about that?
Categories: [Undisturbed, Few interruptions 1 to 2 times, Frequent interruptions 3+ times, Very fragmented sleep]
Answer: I wake up all the time. I don't know why. It's like my brain won't shut off. I just lie there staring at the ceiling.

4. Thoughts Before Sleep
Question: What kind of thoughts do you usually have when you’re trying to fall asleep? Are they calm and relaxing, or do you find yourself worrying about things?
Categories: [excited, hopeful, anxious, worried, blank, forgetful, planning, strategizing]
Answer: My mind just races. I think about everything that went wrong that day, and then I start worrying about tomorrow. It's a never-ending cycle.

5. Thoughts After Waking Up
Question: How do you typically feel when you wake up in the morning? Refreshed and ready to start the day, or tired and groggy?
Categories: [Refreshed, Tired, Hurried, Concerned]
Answer: I feel like a zombie. I don't want to get out of bed. Everything feels heavy and pointless.

6. Kind of Dreams
Question: Can you tell me a bit about your dreams? Do you remember them often? Are they usually pleasant, or do you have nightmares or disturbing dreams?
Categories: [Pleasant, Neutral, Nightmares, Vivid, Lucid, Recurring]
Answer: I don't remember my dreams. Or maybe I do, but they're just a blur. Nothing important.


Extract relevant phrases from the answer and assign categories for each answer:
"""

In [5]:
response = model.completion(prompt)
output = parse_llm_output(response)
print(response)
# print(output)

1. Noise during sleep 
[very noise]
<constant hum in my head>

2. Temperature During Sleep
[too cold]
<I'm usually cold, even when the heat is on, Nothing seems to warm me up>

3. Sleep Continuity
[Frequent interruptions 3+ times, Very fragmented sleep]
<wake up all the time, brain won't shut off>

4. Thoughts Before Sleep
[anxious, worried]
<mind just races, think about everything that went wrong that day, worrying about tomorrow>

5. Thoughts After Waking Up
[Tired, Concerned]
<feel like a zombie, don't want to get out of bed, everything feels heavy and pointless>

6. Kind of Dreams
[Neutral]
<I don't remember my dreams, or maybe I do, but they're just a blur>


In [6]:
output

{'1. Noise during sleep': (['very noise'], ['constant hum in my head']),
 'Temperature During Sleep': (['too cold'],
  ["I'm usually cold",
   'even when the heat is on',
   'Nothing seems to warm me up']),
 'Sleep Continuity': (['Frequent interruptions 3+ times',
   'Very fragmented sleep'],
  ['wake up all the time', "brain won't shut off"]),
 'Thoughts Before Sleep': (['anxious', 'worried'],
  ['mind just races',
   'think about everything that went wrong that day',
   'worrying about tomorrow']),
 'Thoughts After Waking Up': (['Tired', 'Concerned'],
  ['feel like a zombie',
   "don't want to get out of bed",
   'everything feels heavy and pointless']),
 'Kind of Dreams': (['Neutral'],
  ["I don't remember my dreams", 'or maybe I do', "but they're just a blur"])}

In [8]:
prompt = """We are gathering information about patient's sleep health. This information consist of various parameters each having some categories. A category describes the broad state of patient's sleep health in terms of the parameter. 

Your goal is to make (reason, activity) pairs. An activity should be created such that it somehow changes the state of parameter from on category to another category. 
Activities should be related to the topic given, for example if the topic is "yoga" then activity will have to contain some asanas that comes under yoga.

For example, suppose the category of parameter "thoughts before sleep" is "worried". This means that patient is worried about something before going to sleep. An activity like "go for a walk" could make the patient less "worried" and more "calm". This is how we can change the category of one parameter to other parameter by suggesting some activity.

Output Instructions:
    - Always generate a pair of reason and activity.
    - Always enclose reason in left angle bracket < and right angle bracket >.
    - Always enclose activity in left square bracket [ and right square bracket ].
    - Activity should be gentle enough to be done by an average person.
    - Reason should not be very generic, and be informative about the activity.

Given a parameter, current category and target category, suggest a (reason, activity) pair that changes the current category into target category.

Parameter: {parameter}
Current Category: {current_category}
Target Category: {target_category}
Topic: {topic}
Output:
"""

In [13]:
parameter = "Thoughts before sleep"
current_category = "Worried"
target_category = "Calm"
topic = "Pranayama/Breathing exercises"

response = model.completion(prompt.format(parameter=parameter, current_category=current_category, target_category=target_category, topic=topic))
response

'<Practicing slow and deep breathing can calm the mind and reduce worries before sleep.> [Perform Bhastrika Pranayama for 5 minutes, followed by 5 minutes of Anuloma Viloma Pranayama to slow down your breathing and calm your mind.]'

In [14]:
import re
def extract_strings(input_string, encloser):
    enclosers = {"(": ")", "[": "]", "{": "}", "<": ">"}

    if encloser not in enclosers:
        raise ValueError("Invalid encloser. Supported enclosers are (, [, {, and <")

    closing_encloser = enclosers[encloser]

    escaped_open = re.escape(encloser)
    escaped_close = re.escape(closing_encloser)

    pattern = f"{escaped_open}([^{escaped_open}{escaped_close}]*){escaped_close}"
    matches = re.findall(pattern, input_string)

    result = []
    for match in matches:
        result.append(match)

    return result

In [15]:
extract_strings(response, "[")

['Perform Bhastrika Pranayama for 5 minutes, followed by 5 minutes of Anuloma Viloma Pranayama to slow down your breathing and calm your mind.']

In [16]:
extract_strings(response, "<")

['Practicing slow and deep breathing can calm the mind and reduce worries before sleep.']